# Import Modules

In [3]:
import numpy as np
import pandas as pd

import utils

# Load Data

In [4]:
target_vegetable = utils.VEGETABLES[14]

In [5]:
utils.VEGETABLES[14]

'そらまめ'

In [34]:
train_test = pd.read_csv("./data/mapped_train_test.csv")
train_test["date"] = pd.to_datetime(train_test["date"], format="%Y-%m-%d")
weather = pd.read_csv("./data/sorted_mapped_adjusted_weather.csv")
train_test = pd.concat([train_test, weather], axis=1)
train_test = train_test.reset_index(drop=True)

train_test["year"] = train_test.date.dt.year
years = pd.get_dummies(train_test["year"])
train_test = train_test.drop(columns="year")
train_test = pd.concat([train_test, years], axis=1)

train_test["month"] = train_test.date.dt.month
months = pd.get_dummies(train_test["month"])
train_test = train_test.drop(columns="month")
train_test = pd.concat([train_test, months], axis=1)

# train_test["weekday"] = train_test.date.dt.weekday
# weekdays = pd.get_dummies(train_test["weekday"])
# train_test = train_test.drop(columns="weekday")
# train_test = pd.concat([train_test, weekdays], axis=1)

areas = pd.get_dummies(train_test["area"])
train_test = train_test.drop(columns="area")
train_test = pd.concat([train_test, areas], axis=1)

test_size = pd.read_csv("./data/test.csv").shape[0]
train = train_test[:-test_size]
test = train_test[-test_size:]

train = utils.get_target_values(train, target_vegetable)
test = utils.get_target_values(test, target_vegetable)

train_loader, train, test, ss = utils.preprocess_data_submit(train, test)

# Set Free Params

In [35]:
future = test.shape[0]
num_epochs = 70
learning_rate = 0.001
weight_decay = 1e-3

# Training

In [36]:
pred_y = utils.pipeline_rnn_submit(train_loader, train, test, future=future, num_epochs=num_epochs,
                                   lr=learning_rate, weight_decay=weight_decay)

pred_y = pred_y.cpu().detach().numpy()
pred_y = np.concatenate([pred_y.reshape(-1, 1), test[:, 1:]], axis=1)
pred_y[:, :6] = ss.inverse_transform(pred_y[:, :6])

In [37]:
pred_y[:, 0].tolist()

[2633.9907076626478,
 2624.5487355393684,
 2570.2495897482886,
 2488.6321461426373,
 2377.2372333503777,
 2293.601581591989,
 2166.2367492399053,
 2065.35662683659,
 1957.9575316258063,
 1857.7252974886462,
 1769.2105888690812,
 1731.015438013705,
 1717.7673526653252,
 1713.9527237141633,
 1714.6473041794097,
 1717.320523056961,
 1720.5616083959637,
 1720.6784441575273,
 1731.1664502225287,
 1735.6596879216129]